In [ ]:
label = [[1, 'First Party Collection/Use'], 
              [2, 'Third Party Sharing/Collection'], 
              [3, 'User Choice/Control'], 
              [4, 'User Access, Edit and Deletion'], 
              [5, 'Data Retention'],
              [6, 'Data Security'],
              [7, 'Policy Change'], 
              [8, 'Do Not Track'],
              [9, 'International and Specific Audiences'],
              [10, 'Introductory/Generic'],
              [11, 'Privacy contact information'],
              [12, 'Privacy contact information']]

In [ ]:
import os
import json
import csv
import pandas as pd
import nltk
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import numpy as np
from sklearn.metrics import classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def readPolicyFile(fileLocation):
    policySegments = []
    for filename in os.listdir(fileLocation):
        absFilename = "{}/{}".format(fileLocation,filename)
        with open(absFilename) as csv_file:
            #print absFilename
            categoryId = 0
            csv_reader = csv.reader(csv_file, delimiter=',')
            for row in csv_reader:
                if row[5] == "First Party Collection/Use":
                    categoryId = 1
                elif row[5] == "Third Party Sharing/Collection":
                    categoryId = 2
                elif row[5] == "User Choice/Control":
                    categoryId = 3
                elif row[5] == "User Access, Edit and Deletion":
                    categoryId = 4
                elif row[5] == "Data Retention":
                    categoryId = 5
                elif row[5] == "Data Security":
                    categoryId = 6
                elif row[5] == "Policy Change":
                    categoryId = 7
                elif row[5] == "Do Not Track":
                    categoryId = 8 
                elif row[5] == "International and Specific Audiences":
                    categoryId = 9
                elif row[5] == "Introductory/Generic":
                    categoryId = 10
                elif row[5] == "Privacy contact information":
                    categoryId = 11
                elif row[5] == "Practice not covered":
                    categoryId = 12
                else:
                    continue
                    
                policySegment = ''
                jsonData=json.loads(row[6])
                for (k, v) in jsonData.items():
                    for (k, v) in v.items():
                        if k == 'selectedText':
                            policySegment = ''.join(v)
                
                policySegments.append([policySegment, categoryId, row[5]])
            #print policySegments
            #print policySegments
    df = pd.DataFrame(policySegments, columns = ['text', 'label', 'label_name'])
    return df

In [ ]:
def cleanDocs(dataFrame):
    cleanNull = dataFrame[df.text != 'null'].reset_index(drop=True)
    stop = set(stopwords.words('english'))
    exclude = set(string.punctuation) 
    lemma = WordNetLemmatizer()
    clean_docs = []
    bigram_docs = []
    for index, entry in enumerate(cleanNull['text']):
        stop_free = " ".join([i for i in entry.lower().split() if i not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        digit_free = [word for word in punc_free.split() if not word.isdigit() and len(word) > 2]
        normalized = " ".join(lemma.lemmatize(word) for word in digit_free)
        nouns = [word[0] for word in nltk.pos_tag(normalized.split()) if word[1] == 'NN' or word[1] == 'VB']
        cleanNull.loc[index,'text_final'] = str(nouns)

	#bigram_transformer = phrases.Phrases(clean_docs)
	
	#for doc in bigram_transformer[clean_docs]:
	#		bigram_docs.append(doc)
    cleanEmpty = cleanNull[cleanNull.text_final != '[]']
    return cleanEmpty

In [ ]:
def loadTestDataset(fileName):
    df = pd.read_csv(fileName)
    return df

In [ ]:
def buildModel(Corpus):
    Train_data, Test_data, Train_label, Test_label = train_test_split(Corpus['text_final'],Corpus['label'],test_size=0.3)
    #Encoder = preprocessing.LabelEncoder()
    #Train_label = Encoder.fit_transform(Train_label)
    #Test_label = Encoder.fit_transform(Test_label)
    Tfidf_vect = TfidfVectorizer(max_features=5000)
    Tfidf_vect.fit(Corpus['text_final'])
    Train_data_Tfidf = Tfidf_vect.transform(Train_data)
    Test_data_Tfidf = Tfidf_vect.transform(Test_data)
    
#     print(Tfidf_vect.vocabulary_)
    
    # fit the training dataset on the NB classifier
#     Naive = MultinomialNB()
#     Naive.fit(Train_data_Tfidf,Train_label)
#     # predict the labels on validation dataset
#     predictions_NB = Naive.predict(Test_data_Tfidf)
#     # Use accuracy_score function to get the accuracy
#     print(classification_report(Test_label, predictions_NB))
#     print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_label)*100)
    
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVM = SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    SVM.fit(Train_data_Tfidf,Train_label)
    # predict the labels on validation dataset
    predictions_SVM = SVM.predict(Test_data_Tfidf)
    #print Test_data_Tfidf
    #print predictions_SVM
    # Use accuracy_score function to get the accuracy
    print(classification_report(Test_label, predictions_SVM))
    print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_label)*100)
    
    
    return SVM

In [ ]:
def predictLable(model, corpus):
    print (corpus)
    Tfidf_vect = TfidfVectorizer(max_features=5000)
    Tfidf_vect.fit(corpus['text_final'])
    webPolicy_TFidf = Tfidf_vect.transform(corpus['text_final'])
    webPolicyPredition = model.predict(webPolicy_TFidf)
    
    return webPolicyPredition;

In [ ]:
def mergeData(corpus, predictedResult):
    labels = [[1, 'First Party Collection/Use'], 
              [2, 'Third Party Sharing/Collection'], 
              [3, 'User Choice/Control'], 
              [4, 'User Access, Edit and Deletion'], 
              [5, 'Data Retention'],
              [6, 'Data Security'],
              [7, 'Policy Change'], 
              [8, 'Do Not Track'],
              [9, 'International and Specific Audiences'],
              [10, 'Introductory/Generic'],
              [11, 'Privacy contact information'],
              [12, 'Privacy contact information']]
    
    dfLabel = pd.DataFrame(labels, columns=['label', 'discription'])
    dfPredictedResult = pd.DataFrame(predictedResult)
    dfContact = pd.concat([corpus, dfPredictedResult], axis=1)
    dfContact.columns = ['topic_number', 'corpus', 'label'] 
    return pd.merge(dfContact, dfLabel, on='label')

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
np.random.seed(500)
fileLocation = '/content/drive/MyDrive/annotations'

df = readPolicyFile(fileLocation)
Corpus = cleanDocs(df)
print (Corpus['label_name'].unique())
Corpus.to_csv('clean_OOP-115_policy_corpus.csv', index=False)
model = buildModel(Corpus)

# webPolicyCorpus = loadTestDataset('topic_10_only_nouns_n_verb_run_1.csv')
# gdprPolicyCorpus = loadTestDataset('topic_10_gdpr_only_nouns_n_verb_run_1.csv')

# webPolicyPrediction = predictLable(model, webPolicyCorpus)
# gdprPolicyPrediction = predictLable(model, gdprPolicyCorpus)

# mergeData(webPolicyCorpus, webPolicyPrediction)


['First Party Collection/Use' 'User Choice/Control'
 'Third Party Sharing/Collection' 'Data Security'
 'International and Specific Audiences' 'Policy Change'
 'User Access, Edit and Deletion' 'Data Retention' 'Do Not Track']
              precision    recall  f1-score   support

           1       0.80      0.91      0.85      2392
           2       0.88      0.79      0.83      1387
           3       0.64      0.74      0.68       442
           4       0.69      0.41      0.52       227
           5       0.81      0.23      0.35        97
           6       0.92      0.72      0.81       310
           7       0.87      0.71      0.78       172
           8       1.00      0.93      0.96        29
           9       0.90      0.87      0.88       293

    accuracy                           0.81      5349
   macro avg       0.83      0.70      0.74      5349
weighted avg       0.82      0.81      0.81      5349

SVM Accuracy Score ->  81.30491680687979


In [ ]:
"""
import pandas as pd
d={"text":["personal and other information that may be collected when you interact with the Barnes & Noble enterprise,"]}
corpus=pd.DataFrame(d)
corpus.to_csv("out.csv",index=False)
"""

'\nimport pandas as pd\nd={"text":["personal and other information that may be collected when you interact with the Barnes & Noble enterprise,"]}\ncorpus=pd.DataFrame(d)\ncorpus.to_csv("out.csv",index=False)\n'

In [ ]:
corpus = loadTestDataset('/content/input.csv')
print(corpus)

                                                    text
0              to help customize your TED.com experience
1         certain information from your Facebook profile
2         certain information from your Facebook profile
3      required for TED Conference registration or ot...
4      required for TED Conference registration or ot...
...                                                  ...
17822                    we will notify parents by email
17823  Personally Identifiable Information on communi...
17824  Social Media Features are either hosted by a t...
17825  Social Media Features are either hosted by a t...
17826                send a gift certificate or playlist

[17827 rows x 1 columns]


In [ ]:
k= predictLable(model, cleanDocs(corpus))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


                                                    text                                         text_final
0              to help customize your TED.com experience                ['help', 'customize', 'experience']
1         certain information from your Facebook profile             ['information', 'facebook', 'profile']
2         certain information from your Facebook profile             ['information', 'facebook', 'profile']
3      required for TED Conference registration or ot...          ['conference', 'registration', 'service']
4      required for TED Conference registration or ot...          ['conference', 'registration', 'service']
...                                                  ...                                                ...
17801                    we will notify parents by email                                ['parent', 'email']
17802  Personally Identifiable Information on communi...  ['information', 'community', 'message', 'board...
17803  Social Media Features

In [ ]:
l=list(k)

In [ ]:
freq = {}

for item in l:
  if item in freq:
    freq[item] += 1
  
  else:
    freq[item] = 1

print(freq)

{1: 8944, 3: 1718, 2: 4275, 6: 873, 9: 910, 7: 460, 4: 442, 5: 103, 8: 81}


In [46]:
list = []
for i in range(1, 13):
  if i == 3:
    pass
  elif i == 4:
    pass
  else:
    try:
      print(freq[i])
      list.append(freq[i])
    except:
      pass

list

8944
4275
103
873
460
81
910


[8944, 4275, 103, 873, 460, 81, 910]

In [ ]:
list

[8944, 4275, 103, 873, 460, 81, 910]

In [ ]:
val = freq[3] + freq[4]
if freq[1] > freq[2]:
  if freq [6] > freq[5]:
    if any(y>val for y in list ):
      print("A")
    elif any(y<=val for y in list ):
      print("B")
elif freq[1] <=freq[2]:
  if freq [6] <= freq[5]:
    if any(y>=val for y in list ):
      print("C")
    elif any(y<=val for y in list ):
      print("D")
  elif freq[6] > freq[5]:
    if any(y>=val for y in list ):
      print("D")
    if any(y<=val for y in list ):
      print("D")

A
